##### Outputs from this notebook is saved in `../data/files_list` folder

In [1]:
import hyp3_sdk as sdk
from hyp3_sdk import HyP3
import getpass
from datetime import datetime
import os
import sys
from itertools import product as itrprod

sys.path.insert(1, "../")
from utils import *
import re

In [2]:
AN_POLYGONS = [
    "POLYGON((67.11977875914101 -73.13320972913671,68.76600937571335 -73.2322558460891,69.05248236149063 -72.74522842085329,67.48159030357469 -72.65858592955519,67.11977875914101 -73.13320972913671))",
    "POLYGON((71.31571253623807 -70.6602525972592,72.75518985367877 -70.71429083301972,73.00669031483929 -70.19279799544364,71.50559736489964 -70.15970191893189,71.31571253623807 -70.6602525972592))",
]

In [2]:
SAVE_DIR = "../data/files_list"
os.makedirs(SAVE_DIR, exist_ok=True)

In [9]:
# false color is B12, B11, B4 bands 
S2_DATA_DIR = "/g/data/fj7/Copernicus/Sentinel-2/MSI/"
# S2_AOI_LIST_AN = [
#     "N0511_R118_T42DXG",
#     "N0511_R056_T42CVE",
# ]
S2_AOI_LIST_AN = [
    # "R118_T42DXG",
    # re.compile("N05\\d+_R(056|061|118)_T42CVE"),
    # re.compile("N05\\d+_R(056|061|118)_T42DXJ"),
    # re.compile("N05\\d+_R074_T51KYT")  # this is actually AU in WA
    # re.compile("N05\\d+_R061_T42DWK"),
    re.compile("N05\\d+_R032_T43DFD"),
]
S2_QUERY_YEARS_AN = [str(i) for i in range(2023, 2025)]
S2_QUERY_PRODS = ["L1C", "L2A"]
MONTH_RANGE = ["{:02d}".format(m) for m in range(1, 13)]
S2_CASE_LIST_AN = list(itrprod(S2_QUERY_PRODS, S2_QUERY_YEARS_AN, MONTH_RANGE))

S2_AN_FILENAME = "s2_au_wa.csv"

In [25]:
# Alternative way, using SARA server API, faster but only returns names not full path on local NCI storage

# scene_names = get_sentinel_filenames(
#     polygon_list=AN_POLYGONS,
#     years=["2023", "2024"],
#     is_poly_bbox=False,
#     satellite="S2",
#     instrument="MSI",
#     output_file_path="",
#     # return_query_only = True,
#     includes = S2_AOI_LIST_AN,
# )

In [10]:
scenes_dict = await find_files_for_aios_async(
    S2_CASE_LIST_AN, S2_DATA_DIR, S2_AOI_LIST_AN
)
for key in scenes_dict.keys():
    print(f"{key} => {len(scenes_dict[key])}")

Retrieving files for /g/data/fj7/Copernicus/Sentinel-2/MSI/L2A/2024/2024-12
filtering files for re.compile('N05\\d+_R032_T43DFD')
re.compile('N05\\d+_R032_T43DFD') => 86


In [14]:
os.makedirs("temp", exist_ok=True)
save_file_list(scenes_dict, os.path.join("temp", S2_AN_FILENAME))
s2_an_df = pd.read_csv(os.path.join("temp", S2_AN_FILENAME), names=["ID", "Path"])
shutil.rmtree("temp")
scenes_dict_filtered = get_scenes_dict(s2_an_df, is_s1=False)

In [15]:
s2_id = list(scenes_dict_filtered.keys())[0]
s2_scene_files = scenes_dict_filtered[s2_id]

In [17]:
# get the Jan 2023 for S2 L1C
for i, s2_scene in enumerate(s2_scene_files):
    print(f"{i + 1} of {len(s2_scene_files)}; Extracting {s2_scene}", end="\r")
    with ZipFile(s2_scene) as f:
        f.extractall(f"../data/inputs/{s2_id}")

In [18]:
p = re.compile("_\\d+m.jp2")
ref_image_dir = os.path.join("../data/inputs/", s2_id)
ref_tci_files = list(
    filter(
        lambda f: "TCI" in f,
        glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
    )
)
os.makedirs("../data/inputs/temp", exist_ok=True)
for file in ref_tci_files:
    shutil.copy(file, os.path.join("../data/inputs/temp", os.path.basename(file)))
shutil.rmtree(ref_image_dir, ignore_errors=True)
s2_id = s2_id.split("_")[-1].replace("')", "")
ref_image_dir = os.path.join("../data/inputs/", s2_id)
os.makedirs(ref_image_dir, exist_ok=True)
copied_files = glob.glob("../data/inputs/temp/**")
for file in copied_files:
    if len(re.findall(p, file)) == 0:
        prod_dir = "L1C"
    else:
        prod_dir = "L2A"
    if not os.path.isdir(os.path.join(ref_image_dir, prod_dir)):
        os.makedirs(os.path.join(ref_image_dir, prod_dir), exist_ok=True)
    shutil.copy(file, os.path.join(ref_image_dir, prod_dir, os.path.basename(file)))
shutil.rmtree("../data/inputs/temp", ignore_errors=True)

In [ ]:
# find scenes
s2_id = list(scenes_dict.keys())[0]
s2_scene_files = scenes_dict[s2_id]

# get the Jan 2023 for S2 L1C
s2_scene = s2_scene_files[1]
with ZipFile(s2_scene) as f:
    f.extractall(f"../data/inputs/{s2_id}")

ref_image_dir = os.path.join("../data/inputs/", s2_id)
ref_tci_files = list(
    filter(
        lambda f: "TCI" in f,
        glob.glob(f"{ref_image_dir}/*/GRANULE/*/IMG_DATA/**", recursive=True),
    )
)
ref_tci_files = list(filter(lambda f: S2_QUERY_PRODS[0] in f, ref_tci_files))
if len(ref_tci_files) > 1:
    ref_image = [f for f in ref_tci_files if f.endswith("_10m.jp2")][0]
else:
    ref_image = ref_tci_files[0]

ref = rasterio.open(ref_image)
ref_bounds = ref.bounds
ref_crs = ref.crs

ref_proj = Proj(**ref_crs.data)

west, south = ref_proj(ref_bounds.left, ref_bounds.bottom, inverse=True)
east, north = ref_proj(ref_bounds.right, ref_bounds.top, inverse=True)

bbox = f"{west},{south},{east},{north}"

os.makedirs("../data/asf/sub1", exist_ok=True)
with open(f"../data/asf/sub1/bbox.txt", "w") as f:
    f.write(f"{bbox}\n")
    f.write(f"{s2_id}\n")

In [ ]:
project_name = f"AN-coreg-test-20m-{datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"
print(project_name)

with open(f"../data/asf/sub1/bbox.txt", "r") as f:
    bbox = f.readline().strip()
    s2_id = f.readline().strip()

s1_scene_files = get_sentinel_filenames(
    [bbox], ["2023", "2024"], f"../data/asf/sub1/scenes.txt", ["SLC"]
)
# get the Dec 2024 for S1 GRD
scene = s1_scene_files[0]

In [3]:
uid = getpass.getpass("Enter your EathData user id?")
pswd = getpass.getpass("Enter your EathData password?")
hyp3 = HyP3(username=uid, password=pswd)

In [ ]:
download_folder = f"../data/asf/{s2_id}/sub1/"

rtc_job = sdk.Batch()
rtc_job = hyp3.submit_rtc_job(
    os.path.splitext(os.path.basename(scene))[0],
    include_dem=True,  # include dem in final product
    include_inc_map=True,  # include dem map in final product
    include_rgb=True,  # include rgb img in final product
    include_scattering_area=True,  # include scat area in final product
    name=project_name,
    resolution=20,
    dem_name="copernicus",
    radiometry="gamma0",
    speckle_filter=True,
)
print(rtc_job)

# watch job
rtc_job = hyp3.find_jobs(name=project_name)
rtc_job = hyp3.watch(rtc_job)

# download when done
rtc_file = rtc_job.download_files(download_folder)

os.makedirs(f"../data/inputs/{s2_id}/sub1", exist_ok=True)
with open(f"../data/inputs/{s2_id}/sub1/rtc.txt", "w") as f:
    f.write(f"{str(rtc_file[0])}")